# Replicate Regression Output

* **Author**: Cameron Raymond
* **Contact**: cameron.raymond@hey.com
* **Date**: Sept. 14, 2022

Replicating Table 3 (regression results) from the paper, *Automated vs. Manual Case Investigation and Contact Tracing for Pandemic Surveillance: Evidence from a Stepped Wedge Cluster Randomized Trial*.

Note: while this notebook replicates our main results with aggregate data, the original paper performed analyses at the individual level. 


In [1]:
library(tidyverse)
library(stargazer)
library(ivreg)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




In the original paper we analyze the data at the individual level. Our ITT analysis implements a two-way fixed effects estimator with standard errors clustered at the ZIP Code level. Our IV analysis uses the randomization protocol as an instrument for the treatment ultimately assigned.

To adapt our analyses using the aggregate panel data we no longer cluster our standard errors since observations are aggregated to the ZIP Code level anyway. The ITT analyses remain very similar. The IV analysis uses the ZIP Code randomization as an instrument for the percentage of individuals assigned to the treatment. In addition, since there is quite a bit of variance in how many observations are recorded in each week, we weight observations by the number of participants.

In [7]:
panel_data <- read_csv('./panel_data.csv') %>%
    mutate(
        svi_label = svi_label %>%  as.factor() %>% relevel(ref = '[50,80) SVI'),
        condition = condition %>% as.factor() %>% relevel(ref = 'Manual CICT'),
        n_cases = n_assigned_automated + n_assigned_manual,
        # Use randomization as an instrument for the % of those actually assigned CICT
        perc_assigned_automated = n_assigned_automated / n_cases,
        # Weighted average of the two completion rates (manual and automated)
        overall_completion_rate = (mean_perc_filled_assigned_automated * n_assigned_automated + mean_perc_filled_assigned_manual * n_assigned_manual) / (n_assigned_automated + n_assigned_manual)) %>%
    select(week, crossover, svi_label, Zip, condition, n_cases, perc_assigned_automated, overall_completion_rate)

Rows: 231 Columns: 31
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (2): svi_label, condition
dbl  (28): Zip, crossover, n_assigned_automated, n_assigned_manual, mean_per...
date  (1): week

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [9]:
itt.md <- lm(
    overall_completion_rate ~ condition + svi_label + as.factor(week) + as.factor(Zip),
    weights = n_cases,
    data = panel_data)

iv_reg.md <- ivreg(
    overall_completion_rate ~ perc_assigned_automated + svi_label + as.factor(week) + as.factor(Zip) | . - perc_assigned_automated + condition,
    weights = n_cases,
    data = panel_data)

stargazer(
    itt.md,
    iv_reg.md,
    keep = c("condition","perc_assigned_automated","svi_label"),
    order = c("condition","perc_assigned_automated","svi_label"),
    covariate.labels = c('Randomized to Automated','Assigned to Automated','High SVI'),
    dep.var.labels = c("Percent Fields Known"),
    align = TRUE,
    type = "text")


                                        Dependent variable:         
                               -------------------------------------
                                       Percent Fields Known         
                                         OLS            instrumental
                                                          variable  
                                         (1)                (2)     
--------------------------------------------------------------------
Randomized to Automated               -0.018***                     
                                       (0.006)                      
                                                                    
Assigned to Automated                                    -0.255***  
                                                          (0.068)   
                                                                    
High SVI                                0.001              -0.002   
                                 